# **Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
# import keras_tuner as kt
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, Huber
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import InputLayer, Dense, LSTM, Dropout,GRU
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import seaborn as sns
import os
import datetime
import cv2

# **Reading CSV, Creat DataFrame**

In [3]:
# base_dir = '/content/drive/MyDrive/MTech Internship/interpolated_file.csv'
folder_path = (r"/content/drive/MyDrive/MTech Internship/interpolated_file.csv")
df = pd.read_csv(folder_path)
# print(df.head(20))

df['date_time'] = pd.to_datetime(df['date_time'], format = 'mixed')
df = df[(df['date_time'].dt.time >= pd.to_datetime('7:00').time()) &
                  (df['date_time'].dt.time <= pd.to_datetime('19:30').time())]
# print(df.head(100))
print(df.shape)
# print(df.iloc[50:100])
# TESTING

folder_path_t = (r"/content/drive/MyDrive/MTech Internship/adani_guj_sp_loc_data_2024_Test.csv")
df_t = pd.read_csv(folder_path_t)
print(df_t.head(20))

df_t['date_time'] = pd.to_datetime(df_t['date_time'], format = 'mixed')
df_t = df_t[(df_t['date_time'].dt.time >= pd.to_datetime('7:00').time()) &
                  (df_t['date_time'].dt.time <= pd.to_datetime('19:30').time())]
# df_t = df_t.head(260)
# print(df_t)

(17186, 2)
           date_time        ghi
0    04/01/2024 7:00   89.90500
1    04/01/2024 7:30  157.20000
2    04/01/2024 8:00  177.67000
3    04/01/2024 8:30  307.31833
4    04/01/2024 9:00  352.46167
5    04/01/2024 9:30  597.24164
6   04/01/2024 10:00  725.10500
7   04/01/2024 10:30  814.79170
8   04/01/2024 11:00  739.87164
9   04/01/2024 11:30  864.28330
10  04/01/2024 12:00  963.52167
11  04/01/2024 12:30  976.79000
12  04/01/2024 13:00  974.26330
13  04/01/2024 13:30  930.86334
14  04/01/2024 14:00  913.39830
15  04/01/2024 14:30  854.14500
16  04/01/2024 15:00  802.99664
17  04/01/2024 15:30  712.96000
18  04/01/2024 16:00  612.42500
19  04/01/2024 16:30  497.53168


# **Preprocessing**

In [4]:
starting_timestamp = "2022/01/01 0:00"
ending_timestamp = "2023/12/31 23:30"
array_range = pd.date_range(start=starting_timestamp, end=ending_timestamp, freq='30T')

array_timestamps = []

start_time = pd.to_datetime("7:00 AM").time()
end_time = pd.to_datetime("7:00 PM").time()
max_end_time = pd.to_datetime("7:30 PM").time()

for timestamp in array_range:
    if timestamp.time() >= start_time and timestamp.time() <= end_time:
        array_timestamps.append(timestamp)
    elif timestamp.time() > end_time and timestamp.time() <= max_end_time:
        array_timestamps.append(timestamp)


df_date = pd.DataFrame({'date_time':array_timestamps})
df_date.set_index('date_time')

p_df = pd.merge(df_date, df, on ='date_time', how='left')
print(p_df.head(40))

cols = list(p_df)[1]
# print(cols)

df_for_training = p_df[cols].astype(float)
        # print(df_for_training.head(10))
df_for_training = df_for_training.dropna()
# df_for_training.to_csv(os.path.join(base_dir,"interpolated_file.csv"),header=True)
print(df_for_training.head(40))
print(df_for_training.shape)
df_for_training = np.array(df_for_training).reshape(-1,1)




# TESTING

starting_timestamp_t = "01/01/2024 0:00"
ending_timestamp_t = "01/10/2024 23:30"
array_range_t = pd.date_range(start=starting_timestamp_t, end=ending_timestamp_t, freq='30T')

array_timestamps_t = []
start_time_t = pd.to_datetime("7:00 AM").time()
end_time_t = pd.to_datetime("7:00 PM").time()
max_end_time_t = pd.to_datetime("7:30 PM").time()

for timestamp_t in array_range_t:
    if timestamp_t.time() >= start_time_t and timestamp_t.time() <= end_time_t:
        array_timestamps_t.append(timestamp_t)
    elif timestamp_t.time() > end_time_t and timestamp_t.time() <= max_end_time_t:
        array_timestamps_t.append(timestamp_t)

df_date_t = pd.DataFrame({'date_time':array_timestamps_t})
df_date_t.set_index('date_time')
p_df_t = pd.merge(df_date_t, df_t, on ='date_time', how='left')
# print(p_df_t)
cols_t = list(df_t)[1]
# print(cols)
df_for_testing = df_t[cols].astype(float)
        # print(df_for_testing.head(10))
df_for_testing = df_for_testing.dropna()
# df_for_testing.to_csv(os.path.join(base_dir,"interpolated_file.csv"),header=True)
# print(df_for_testing.head(60))
print(df_for_testing.shape)
df_for_testing = np.array(df_for_testing).reshape(-1,1)


<ipython-input-4-4b9728d1e37f>:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  array_range = pd.date_range(start=starting_timestamp, end=ending_timestamp, freq='30T')


             date_time         ghi
0  2022-01-01 07:00:00    0.535000
1  2022-01-01 07:30:00   11.283334
2  2022-01-01 08:00:00   31.163334
3  2022-01-01 08:30:00  160.016660
4  2022-01-01 09:00:00  157.360000
5  2022-01-01 09:30:00  315.043330
6  2022-01-01 10:00:00  448.633330
7  2022-01-01 10:30:00  522.446660
8  2022-01-01 11:00:00  578.741640
9  2022-01-01 11:30:00  660.075000
10 2022-01-01 12:00:00  697.278300
11 2022-01-01 12:30:00  605.298340
12 2022-01-01 13:00:00  696.310000
13 2022-01-01 13:30:00  664.900000
14 2022-01-01 14:00:00  641.723300
15 2022-01-01 14:30:00  576.855000
16 2022-01-01 15:00:00  489.220000
17 2022-01-01 15:30:00  404.913330
18 2022-01-01 16:00:00  318.401670
19 2022-01-01 16:30:00  218.431670
20 2022-01-01 17:00:00  123.950000
21 2022-01-01 17:30:00   45.461666
22 2022-01-01 18:00:00    5.376666
23 2022-01-01 18:30:00    0.000000
24 2022-01-01 19:00:00    0.000000
25 2022-01-01 19:30:00    0.000000
26 2022-01-02 07:00:00    0.518333
27 2022-01-02 07:30:

<ipython-input-4-4b9728d1e37f>:42: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  array_range_t = pd.date_range(start=starting_timestamp_t, end=ending_timestamp_t, freq='30T')


In [5]:
max_value = p_df['ghi'].max()
min_value = p_df['ghi'].min()
print(max_value,min_value)

1118.2317 0.0


# **Handling Missing Values**


In [ ]:

def missingValUsingMedian(missing_val_date):

    print("Neighbouring values not available")
    print("Find previous 3 days values and perform median operation")

    dt = missing_val_date
    prev_dates_lst = []
    prev_values_lst = []
    for k in range(3):
        dt1 = dt - datetime.timedelta(days=k+1)
        dt_str = dt1.strftime("%Y-%m-%d %H:%M")
        prev_dates_lst.append(dt_str)
        value = df[df['date_time'] == dt_str][col].values[0]
        if value != None:
            print(value)
            prev_values_lst.append(value)
    print(prev_values_lst)
    missing_val = np.nanmedian(prev_values_lst)
    print(missing_val)

    return missing_val

for col in p_df.columns:
    print(col)
    for i, row in p_df.iterrows():
        dt = row[0]
        if i==0:
            pass
        else:
            if pd.isnull(p_df.loc[i][col]):
                print("Value null for dt::",row[0])
                if not pd.isnull(p_df.loc[i-1][col]):
                    if not pd.isnull(p_df.loc[i+1][col]):
                        p_df.loc[i,col] = (p_df.loc[i-1][col] + p_df.loc[i+1][col])/2
                        print(p_df.loc[i, col])
                        # p_df[col].replace(p_df.loc[i,col], new_val, inplace = True)
                        print("Replaced Value:", p_df.loc[i, col])

                    elif not pd.isnull(p_df.loc[i+2][col]):
                        print("here--1")
                        p_df.loc[i,col] = (p_df.loc[i-1][col] + p_df.loc[i+2][col])/2
                        # p_df.replace(p_df.loc[i,col], new_val, inplace = True)
                    else:
                        replaced_val = missingValUsingMedian(dt)
                        print("missing Values::",replaced_val)
                        p_df.loc[i,col]= replaced_val
                        p_df.replace(p_df.loc[i,col], replaced_val, inplace = True)
                        print("Replaced Value:", p_df.loc[i, col])
                else:
                    replaced_val = missingValUsingMedian(dt)
                    print("missing Values::",replaced_val)
                    p_df.loc[i,col]= replaced_val
                    # p_df.replace(p_df.loc[i,col], replaced_val, inplace = True)
                    print("Replaced Value:", p_df.loc[i, col])



# **Making Windows**

In [6]:
e =60   # epochs
n_past=26*7   # Number of past days we want to use to predict the future.
n_future = 26*2   # Number of days we want to look into the future based on the past days.
n_features = 1   # Single variate

In [7]:

# Empty lists to be populated using formatted training data
dataX, dataY = [], []

for i in range(0,len(df_for_training)-n_past-n_future):
    a = df_for_training[i:(i+n_past), 0]
    dataX.append(a)
    b = df_for_training[i + n_past:i + n_past + n_future, 0]
    dataY.append(b)
dataX, dataY = np.array(dataX), np.array(dataY)
dataX = dataX.reshape((dataX.shape[0], dataX.shape[1], 1))
dataY = dataY.reshape((dataY.shape[0],dataY.shape[1], 1))
# print(dataX)
print(dataX.shape,dataY.shape)
# print(dataY)

# TESTING

dataX_t, dataY_t = [], []
# print(df_for_testing)
for i in range(0,len(df_for_testing)-n_past-n_future):
    a = df_for_testing[i:(i+n_past), 0]
    dataX_t.append(a)
    b = df_for_testing[i + n_past:i + n_past + n_future, 0]
    dataY_t.append(b)
dataX_t, dataY_t = np.array(dataX_t), np.array(dataY_t)

print(dataX_t.shape,dataY_t.shape)
print(dataX_t)
print('**********')
print(dataY_t)


(16952, 182, 1) (16952, 52, 1)
(546, 182) (546, 52)
[[ 89.905     157.2       177.67      ...  38.806667    1.7716666
    0.       ]
 [157.2       177.67      307.31833   ...   1.7716666   0.
   57.358334 ]
 [177.67      307.31833   352.46167   ...   0.         57.358334
  156.105    ]
 ...
 [ 59.93833     4.153333    0.        ... 413.28665   284.27167
   56.363335 ]
 [  4.153333    0.         95.31      ... 284.27167    56.363335
   45.02833  ]
 [  0.         95.31      199.305     ...  56.363335   45.02833
    2.4916666]]
**********
[[ 57.358334  156.105     268.32834   ...  36.055       1.93
    0.       ]
 [156.105     268.32834   390.70834   ...   1.93        0.
   63.923332 ]
 [268.32834   390.70834   515.705     ...   0.         63.923332
  117.40667  ]
 ...
 [ 45.02833     2.4916666   0.        ... 457.385     326.55334
   62.203335 ]
 [  2.4916666   0.         74.87666   ... 326.55334    62.203335
   36.993332 ]
 [  0.         74.87666   160.46666   ...  62.203335   36.993332

# **Scaling**

In [ ]:

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit()
df_for_training_scaled = scaler.transform(dataX)
print(df_for_training_scaled[:10])
counts = np.isnan(df_for_training_scaled).sum()
print(counts)

# df_for_training_scaled = (df_for_training - min_value) /(max_value - min_value)
# print(df_for_training)

df_for_testing_scaled = scaler.transform(dataY)
print(df_for_testing_scaled[:10])
# counts = np.isnan(df_for_training_scaled).sum()

# print(df_for_testing)